### **Librerías**

In [116]:
import os
import math
import time
import smtplib

import pyautogui
import threading
import pywhatkit

import numpy as np
import pandas as pd

from email.mime.application import MIMEApplication
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart

from google.cloud import bigquery

### **BigQuery**

In [117]:
# Proyecto y cliente
project_id='bi-fcom-drmb-local-pe-sbx'
client = bigquery.Client(project=project_id)

# Configuración job
job_config = bigquery.QueryJobConfig()
job_config.query_parameters = (bigquery.ScalarQueryParameter("limit", "INT64", 1000),)

query = """
    SELECT DISTINCT
        dateCreatedAt,
        sellerId,
        sellerName,
        segmento,
        deliveryOrderNumber,
        sellerPhone,
        sellerEmail,
        clientDNI,
        clientName,
        clientPhone,
        clientEmail,
        shipping_fee,
        TotalPrecio,
        monto_total,
        department,
        province,
        district,
        tipo_documento,
        razon_social,
        direccion,
        region,
        provincia,
        persona_fiscal,
        tipo_de_documento,
        identificacion_legal,
        actividad_economica,
        digito_verificador,
        distrito,
        correo_electronico,
        telefono,
        codigo_postal,
        invoiceDate,
        invoiceType,
        invoiceNumber,
        fecha_solicitud,
        ticket_solicitud,
        caseTipification,
        caseStatus,
        fecha_inicio_solicitud,
        fecha_final_solicitud,
        status_comprobante,
        dias_comprobante,
        dias_solicitud,
        n_solicitudes,
        loyaltyPhone,
        EnlaceAAA
    FROM (
        SELECT * FROM `bi-fcom-drmb-local-pe-sbx.Dragonite_SX_KPIs.reporte_comprobantes` 
        WHERE invoiceDate IS NULL
        AND DATE_DIFF(CURRENT_DATE(),datecreatedAt,DAY)<=15
        AND dias_comprobante>0
        AND integrado='No'
        UNION ALL
        SELECT * FROM `bi-fcom-drmb-local-pe-sbx.Dragonite_SX_KPIs.reporte_comprobantes` 
        WHERE ticket_solicitud IS NOT NULL
        AND caseStatus!='Closed'
        AND status_comprobante!='Enviado'
    )
    ORDER BY dateCreatedAt DESC
"""

# Query execution
query_job = client.query(query, job_config=job_config)
query_job.result()
data = query_job.to_dataframe()

c:\Users\gfloress\AppData\Local\Programs\Python\Python39\lib\site-packages\google\auth\_default.py:76: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


### **Variables**

In [118]:
# Correo

correo_usuario = 'sleona@falabella.com'
contrasena = 'Ingreso2024$$'
asuntos_base = [
    'F.COM | 🚨 ENVÍA TUS COMPROBANTES PENDIENTES 🚨'
]

# Columnas

columnas = [
    'sellerId','sellerName',
    'dateCreatedAt','deliveryOrderNumber', 
    'shipping_fee','TotalPrecio','monto_total',
    'tipo_documento', 
    'tipo_de_documento', 'identificacion_legal',
    'razon_social', 
    'region', 'provincia', 'distrito', 'direccion',
    #'correo_electronico', 'telefono',  
    #'persona_fiscal',  
    #'actividad_economica', 'digito_verificador', 
    #'codigo_postal'
]

# Variables iniciales

# NOMBRE, DIRECCION, DNI, FECHA DE EMISION, 

### **Funciones**

In [119]:
"""
def send_email(correo_seller, html_message, seller_email, seller_name, seller_id, seller_phone, seller_link, sender_email, sender_password):
    # Set up the SMTP server
    server = smtplib.SMTP('smtp.office365.com', 587)
    server.starttls()

    # Log in to the SMTP server
    server.login(correo_usuario, contrasena)

    # Set up the message
    msg = MIMEMultipart()
    msg['From'] = 'fcomlogisticasx@falabella.com'
    msg['To'] = correo_seller
    msg['Subject'] = asunto
    msg['CC'] = 'soporteperufcom@falabella.com'

    # Attach the HTML message
    msg.attach(MIMEText(html_message, 'html'))

    # Send the email
    server.sendmail(sender_email, seller_email, msg.as_string())

    # Close the SMTP connection
    server.quit()
"""

"\ndef send_email(correo_seller, html_message, seller_email, seller_name, seller_id, seller_phone, seller_link, sender_email, sender_password):\n    # Set up the SMTP server\n    server = smtplib.SMTP('smtp.office365.com', 587)\n    server.starttls()\n\n    # Log in to the SMTP server\n    server.login(correo_usuario, contrasena)\n\n    # Set up the message\n    msg = MIMEMultipart()\n    msg['From'] = 'fcomlogisticasx@falabella.com'\n    msg['To'] = correo_seller\n    msg['Subject'] = asunto\n    msg['CC'] = 'soporteperufcom@falabella.com'\n\n    # Attach the HTML message\n    msg.attach(MIMEText(html_message, 'html'))\n\n    # Send the email\n    server.sendmail(sender_email, seller_email, msg.as_string())\n\n    # Close the SMTP connection\n    server.quit()\n"

### **Mensajes**

### **Data**

In [123]:
# Leer el registro desde un archivo Excel
data_1 = (
        data
        .query("sellerEmail.str.contains('@')")
        .assign(**{
            'shipping_fee': lambda x: round(pd.to_numeric(x['shipping_fee']),2),
            'TotalPrecio': lambda x: round(pd.to_numeric(x['TotalPrecio']),2),
            'monto_total': lambda x: round(pd.to_numeric(x['monto_total']),2),
            'tipo_de_documento': lambda x: np.where(x['tipo_documento']=='Boleta','DNI',x['tipo_de_documento']),
            'identificacion_legal': lambda x: np.where(x['tipo_documento']=='Boleta',x['clientDNI'],x['identificacion_legal']),
            'razon_social': lambda x: np.where(x['tipo_documento']=='Boleta',x['clientName'],x['razon_social']),
            'region': lambda x: np.where(x['tipo_documento']=='Boleta',x['department'],x['region']),
            'provincia': lambda x: np.where(x['tipo_documento']=='Boleta',x['province'],x['provincia']),
            'distrito': lambda x: np.where(x['tipo_documento']=='Boleta',x['district'],x['distrito'])
        })
)

### **Condicionales**

In [121]:
# Correos


### **Envío de correos**

#### **1. AAA**

In [129]:
# Establecer la conexión con el servidor SMTP de Outlook
server = smtplib.SMTP('smtp.office365.com', 587)
server.starttls()

# Iniciar sesión en el servidor SMTP
server.login(correo_usuario, contrasena)

# Iterar sobre las tiendas únicas en el DataFrame
for seller in data_1['sellerId'].unique():
    
    # Filtrar dataframe por tienda
    df_seller = data_1[data_1['sellerId'] == seller]

    # Obtener datos por tienda
    correo_seller = df_seller['sellerEmail'].iloc[0]
    phone_asignado = df_seller['loyaltyPhone'].iloc[0]
    link_phone = df_seller['EnlaceAAA'].iloc[0]
    sellerName = df_seller['sellerName'].iloc[0]

    # Seleccionar columnas específicas para mostrar en el cuerpo del correo
    df_seller =  (                 
                    df_seller
                    .drop_duplicates()
                    .sort_values(by='dateCreatedAt', ascending=False)
                    .loc[:,columnas]
                    .rename(columns={
                              'sellerName': 'Nombre del seller',
                              'dateCreatedAt': 'Fecha de orden',
                              'deliveryOrderNumber': 'Número de orden',
                              'shipping_fee': 'Valor de envío',
                              'TotalPrecio': 'Valor de productos',
                              'monto_total': 'Valor total',
                              'tipo_documento': 'Tipo de comprobante',
                              'tipo_de_documento': 'Tipo de documento',
                              'identificacion_legal': 'Número de documento',
                              'razon_social': 'Razón social',
                              'region': 'Región',
                              'provincia': 'Provincia',
                              'distrito': 'Distrito',
                              'direccion': 'Dirección'
                    })
    )
    
    # Configurar el mensaje
    msg = MIMEMultipart()
    msg['From'] = 'fcomlogisticasx@falabella.com'
    msg['To'] = correo_seller
    msg['Subject'] = asuntos_base[0].format(seller)
    msg['CC'] = 'soporteperufcom@falabella.com'

    # Crear el cuerpo del correo en formato HTML
    mensaje_html = f"""
      <html>
        <head>
          <style>
            table {{
              border-collapse: collapse;
              width: 100%;
            }}
            th, td {{
              border: 1px solid black;
              padding: 8px;
              text-align: left;
            }}
            th {{
              background-color: #74B700;
              color: white;
              font-size: 16px;
              font-weight: bold;
              text-align: center;
            }}
          </style>
        </head>
        <body>
          <p><h2><strong>Hola {sellerName}, con ID {seller}!</strong> Recuerda subir los comprobantes de los siguientes pedidos: </h2></p>
          
          <br>
          {df_seller.to_html(index=False)}
          <br>
          <p>Para cada orden debes subir la boleta o factura emitida a Falabella Seller Center.</p>
              <ol>
                  <li>Revisa nuestro tutorial de facturación: <a href="https://ayudaseller.falabella.com/s/article/Generacion-de-boletas-y-facturas-Peru">Tutorial de facturación</a>.</li>
                  <li>Revisa nuestro tutorial de carga de documentos: <a href="https://ayudaseller.falabella.com/s/article/Carga-de-documentos-tributarios">Tutorial de carga de comprobantes</a>.</li>
              </ol>
                            
          <p>🚨 Este paso es obligatorio 🚨 de no hacerlo podrías caer en penalidades.</p>
          <p>🚨 Recuerda que los datos de empresa deben coincidir con tus datos de Falabella Seller Center 🚨.</p>

          <p>Podrás revisar tus órdenes con comprobante pendiente en Falabella Seller Center desde <a href= "https://sellercenter.falabella.com/order/invoice#/purchased-order-list">aquí</a>.</p>

          <p>¡Gracias!</p> 
        </body>
      </html>
    """

    # Adjuntar el cuerpo del correo en formato HTML
    msg.attach(MIMEText(mensaje_html, 'html'))

    # Enviar el correo electrónico
    server.sendmail(correo_usuario, correo_seller, msg.as_string())

# Cerrar la conexión con el servidor SMTP
server.quit()

In [ ]:
#<p>🌟 Si necesitas soporte personalizado a través de WhatsApp, contáctanos en el {phone_asignado} o <a href="{link_phone}">ingresando al siguiente link</a>.</p>
        

### **Envío de Whatsapp**

#### **1. AAA**

In [244]:
def send_whatsapp_message(seller_data):
    mensaje_whatsapp = """
        *Hola {sellerName}, Hoy estás en la ruta de recolección*

        Detectamos que tienes órdenes poseen retraso en la entrega, es importante que las entregues hoy al operador logístico.

        Para mayor detalle te hemos enviado un mail al correo *{correo}* con asunto "F.COM | 🚨 HOY ESTAS EN RUTA🚨"

        🚚 Aquí podrás revisar el tutorial de gestión de órdenes: https://youtu.be/lpk1_y9uYDg

        Queremos reforzar contigo el proceso para gestionarlas:

        📦 Para que podamos recolectar tu orden deberás:

        1-	Podrás revisar tus órdenes en falabella seller center desde aquí: https://sellercenter.falabella.com/order
        2-	Recuerda hacer clic en “Listo para despachar” como máximo un día antes de la fecha límite de despacho (antes de las 10p.m.)

        🌟Si necesitas soporte personalizado a través de WhatsApp, contáctanos en el *{Phone}* o haz clic aquí {link_phone}

        ¡Gracias!
    """
    whatsapp_message = mensaje_whatsapp.format(
        sellerName=seller_data['sellerName'],
        correo=seller_data['sellerEmail'],
        Phone=seller_data['sellerPhone'],
        link_phone=seller_data['loyaltyPhone'],
    )
    
    pywhatkit.sendwhatmsg_instantly(seller_data['sellerPhone'], whatsapp_message)

In [245]:
"""
lista = data_5.groupby(['sellerId'],dropna=False).agg(orders=('deliveryOrderNumber','count')).reset_index().sort_values(['orders'], ascending=False)
sellers_lista = lista[lista['orders']>=5]['sellerId'].to_list()
data_5_final = data_5[data_5['sellerId'].isin(sellers_lista)]
"""

In [247]:
"""
# Batch processing
threads = []
for seller_id, seller_data in data_5_final.groupby('sellerId'):
    seller_data = seller_data.iloc[0]  # Assuming each seller has the same data
    thread = threading.Thread(target=send_whatsapp_message, args=(seller_data,))
    threads.append(thread)
    thread.start()
    time.sleep(12)  # Add delay between starting threads

# Wait for all threads to finish
for thread in threads:
    thread.join()
"""